# TP 1 : Classification de cellules tumorales

L'objectif de ce TP est de mettre en application les méthodes de classification vues en cours (Naive Bayes, SVM, LDA, QDA) et de les comparer sur un problème de classification de cellules tumorales. 

## Description des données

Le jeu de données "Breast cancer Wisconsin (Diagnostic) Data Set" (data.csv) contient les informations de 31 features quantitatives sur 569 échantillons provenant de cellules tumorales; la dernière information est un label binaire indiquant si la tumeur est bénigne ("B") ou maligne ("M"). Une description des données est disponibles sur le site UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29). L'objectif est de construire un classifieur permettant de prédire la malignité de la cellule (présence d'un cancer) à partir des 31 features quantitatives. Le jeu de données comporte les colonnes (features) suivantes :


*Identifiant et label*
- id : identifiant unique (integer)
- diagnosis : (M = maligne, B = benigne)

*Mesures quantitatives calculées à partir d'une image médicale de cellules tumorales*
- radius : distance moyenne du noyau cellulaire à la membrane 
- texture : moyenne des valeurs de la nuance de gris dans l'image originale
- perimeter : périmètre moyen des cellules dans l'image
- area : surface moyenne des cellules dans l'image
- smoothness : moyenne de la variation locale dans la taille du rayon (radius)
- compactness : perimeter^2 / area - 1.0
- concavity : sévérité des parties concaves du contour
- concave points : nombre de parties concaves du contour
- symmetry
- fractal dimension : "coastline approximation" - 1

Avec chacune de ces 10 mesures (faites trois fois pour chaque image), on construit 3 features : la moyenne (mean), de l'écart type (se) et la plus grande valeur (worst). Au final, on a pour chaque cellule son ID, le diagnostic, et 30 features quantitatives construites à partir des 10 mesures.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
import pandas as pd
import seaborn as sns

## 1 - Exploration rapide des données

1) Chargez les données, affichez leur taille, un extrait des données et un résumé statistique

In [2]:
df = pd.read_csv(
    # filename
    "data.csv",
)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

18.966666666666665

In [6]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


2) Quel est le type de chaque colonne ? Le jeu de données contient-il des données manquantes ? 

In [8]:
df.isnull().sum()

id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

## 2 - SVM

On va à présent entraîner un SVM pour construire un classifieur de cellules tumorales.

1) Séparez le jeu de données en un ensemble d'entrainement ("training set") et un ensemble de test ("test set")

In [15]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [13]:
# features
X = df.drop(
    labels=[
        "id",
        "diagnosis"
    ],
    axis=1
)

# response
Y = df["diagnosis"]

X_train, X_test, Y_train, Y_test = train_test_split(
    # data
    X, 
    Y,
    # size du test
    test_size=0.2,
    # seed
    random_state=0,
    # 
    shuffle=True,
)

2) Entraînez un SVM linéaire avec paramètre C (goodness-of-fit strength) fixé à 1.

In [16]:
# build
clf = svm.SVC(
    # kernel
    kernel="linear",
    # goodness of fit
    C=1
)

clf.fit(X_train, Y_train)

SVC(C=1, kernel='linear')

3) Proposez un score d'évaluation pour la classification, et appliquez le au classifieur du SVM linéaire

4) Reproduisez les deux questions précédentes avec des SVM à noyaux en utilisant les noyaux RBF et sigmoid

5) Que pouvez-vous conclure des scores obtenus pour ces trois classifieurs ?

6) À l'aide d'une validation croisée, sélectionnez le meilleur paramètre C pour chacun des trois classifieurs précédents, et représentez les valeurs du score pour une grille de C sur une figure

In [3]:
from sklearn.model_selection import cross_val_score
import sys

7) Conclure

## 3 - Naive Bayes, LDA, QDA

Nous allons à présent utiliser d'autres méthodes de classification vues précédemment en cours : le classifieur Naive bayes, l'Analyse Discriminante Linéaire (LDA) et l'Analyse Discriminante Quadratique (QDA).



In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


1) Rappelez en quelques phrases le principe du classifieur Naive Bayes.

2) Entraînez le classifieur Naive Bayes sur le jeu de données et calculer le score associé

3) Rappelez en quelques phrases le principe de LDA et QDA. Quelle est la différence fondamentale entre ces deux méthodes ? (Reprendre le Problème 2 du TD 1 si besoin)

4) Entraînez les classifieurs LDA et QDA sur les données et calculer leur score

## 4 - kNN

On va maintenant appliquer la méthode des k plus proches voisins (kNN, k nearest neighbours)

In [5]:
from sklearn import neighbors


1) Appliquer la méthode kNN pour la classification avec k=5 fixé, et calculer l'erreur correspondante.

2) Appliquer la même méthode pour une grille de k entre 1 et 20, représenter le score sur une figure et conclure sur l'importance du choix de k.

## 5 - Comparaison des méthodes de classification


1) Conclure sur les différentes méthodes de classification pour le jeu de données considéré, à l'aide des questions précédentes 

2) La comparaison des méthodes dépend fortement du score choisi. En pratique, un faux négatif (cancer réel non détecté) est plus grave qu'un faux positif (absence de cancer détectée comme cancer). Comment peut-on adapter le score (et la méthode d'apprentissage) pour prendre cela en compte ?

3) Proposez un score alternatif, et comparez de nouveau les méthodes précédentes. Conclure.
